## Gurobi VRPSPD model

In [1]:
from gurobipy import Model, GRB

# Define model
model = Model("VRPSPD_TwoIndex_Fixed")

# Define problem parameters
nodes = [0, 1, 2, 3, 4]  # 0 is the depot
customers = nodes[1:]  # Exclude depot
vehicles = [0, 1]  # Two vehicles

# Distance matrix (excluding self-loops)
distance = {(i, j): 10 for i in nodes for j in nodes if i != j}  # Only valid distances

capacity = 100  # Vehicle capacity

# Pickup and delivery demands per customer
pickup = {1: 10, 2: 15, 3: 20, 4: 25}  # Pickup demand
delivery = {1: 5, 2: 10, 3: 15, 4: 20}  # Delivery demand

# Decision variables
x = model.addVars(nodes, nodes, vehicles, vtype=GRB.BINARY, name="x")  # Vehicle travel variables
u = model.addVars(nodes, vtype=GRB.CONTINUOUS, name="u")  # Load at each node

# Objective: Minimize total distance traveled
model.setObjective(sum(distance[i, j] * x[i, j, v] for i in nodes for j in nodes if i != j for v in vehicles), GRB.MINIMIZE)

# Constraint: Each customer must be visited exactly once (Fixed self-loops)
for i in customers:
    model.addConstr(sum(x[i, j, v] for j in nodes if i != j for v in vehicles) == 1, name=f"visit_once_{i}")

# Constraint: Each vehicle must leave and return to the depot
for v in vehicles:
    model.addConstr(sum(x[0, j, v] for j in customers) == 1, name=f"leave_depot_{v}")
    model.addConstr(sum(x[i, 0, v] for i in customers) == 1, name=f"return_depot_{v}")

# Constraint: Flow conservation (ensures valid routing, fixed loops)
for v in vehicles:
    for j in customers:
        model.addConstr(sum(x[i, j, v] for i in nodes if i != j) == sum(x[j, k, v] for k in nodes if j != k), name=f"flow_conservation_{j}_{v}")

# Constraint: Prevent sub-tours using Miller-Tucker-Zemlin (MTZ) (Fixed constraints)
for i in customers:
    for j in customers:
        if i != j:
            model.addConstr(u[i] - u[j] + capacity * sum(x[i, j, v] for v in vehicles) <= capacity - pickup[i], name=f"subtour_{i}_{j}")

# Constraint: Vehicle capacity limits
for v in vehicles:
    model.addConstr(sum((pickup[i] + delivery[i]) * sum(x[i, j, v] for j in nodes if i != j) for i in customers) <= capacity, name=f"vehicle_capacity_{v}")

# Solve the model
model.optimize()

# Display Results
if model.status == GRB.OPTIMAL:
    print("\n--- Optimized VRPSPD Solution (Fixed Version) ---")
    for v in vehicles:
        for i in nodes:
            for j in nodes:
                if i != j and x[i, j, v].x > 0.5:  # If vehicle travels between nodes
                    print(f"Vehicle {v} travels from {i} to {j}")
    print(f"\nTotal Cost: {model.objVal}")


Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 30 rows, 55 columns and 192 nonzeros
Model fingerprint: 0x38640f53
Variable types: 5 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+01, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 60.0000000
Presolve removed 0 rows and 11 columns
Presolve time: 0.02s
Presolved: 30 rows, 44 columns, 190 nonzeros
Variable types: 4 continuous, 40 integer (40 binary)

Explored 0 nodes (0 simplex iterations) in 0.04 seconds (0.00 work units)
Thread count was 12 (of 12 available processors)

Solution count 1: 60 

Optimal solution found (toleranc

## Sensitivity Analysis

In [2]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

# Define function to run the VRPSPD model with varying capacity
def run_vrpspd(capacity):
    model = gp.Model("VRPSPD_Sensitivity_Analysis")

    nodes = [0, 1, 2, 3, 4]  # 0 is the depot
    customers = nodes[1:]  
    vehicles = [0, 1]  

    distance = {(i, j): 10 for i in nodes for j in nodes if i != j}  
    pickup = {1: 10, 2: 15, 3: 20, 4: 25}  
    delivery = {1: 5, 2: 10, 3: 15, 4: 20}  

    x = model.addVars(nodes, nodes, vehicles, vtype=GRB.BINARY, name="x")
    u = model.addVars(nodes, vtype=GRB.CONTINUOUS, name="u")  

    model.setObjective(sum(distance[i, j] * x[i, j, v] for i in nodes for j in nodes if i != j for v in vehicles), GRB.MINIMIZE)

    for i in customers:
        model.addConstr(sum(x[i, j, v] for j in nodes if i != j for v in vehicles) == 1)

    for v in vehicles:
        model.addConstr(sum(x[0, j, v] for j in customers) == 1)
        model.addConstr(sum(x[i, 0, v] for i in customers) == 1)

    for v in vehicles:
        for j in customers:
            model.addConstr(sum(x[i, j, v] for i in nodes if i != j) == sum(x[j, k, v] for k in nodes if j != k))

    for i in customers:
        for j in customers:
            if i != j:
                model.addConstr(u[i] - u[j] + capacity * sum(x[i, j, v] for v in vehicles) <= capacity - pickup[i])

    for v in vehicles:
        model.addConstr(sum((pickup[i] + delivery[i]) * sum(x[i, j, v] for j in nodes if i != j) for i in customers) <= capacity)

    model.optimize()

    return model.objVal if model.status == GRB.OPTIMAL else None

# Perform sensitivity analysis by varying vehicle capacity
capacities = [50, 75, 100, 125, 150]
results = [{"Vehicle Capacity": cap, "Total Cost": run_vrpspd(cap)} for cap in capacities]

# Convert results to DataFrame and print
df_results = pd.DataFrame(results)
print(df_results)


Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 30 rows, 55 columns and 192 nonzeros
Model fingerprint: 0x686c4e69
Variable types: 5 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [1e+01, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Presolve removed 3 rows and 30 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores,

### **Conclusion on Sensitivity Analysis Results**

The analysis examined the effect of varying vehicle capacity on the total cost in the VRPSPD model. The results indicate that a vehicle capacity of 50 is insufficient to meet all demands, as the model could not find a feasible solution at this level. This suggests that the minimum capacity required for feasibility is higher than 50.

Once the vehicle capacity reached 75, the total cost stabilized at 60.0, and increasing capacity beyond this point had no impact on cost. This indicates that the routing is already optimized at this level, and additional capacity does not provide further cost benefits. 

From a practical standpoint, a vehicle capacity range of 75 to 100 is sufficient to ensure feasibility while avoiding unnecessary vehicle space. Increasing capacity beyond this level does not improve efficiency, suggesting that the primary limitation is in routing rather than vehicle capacity constraints. 

The key takeaway from this analysis is that optimizing vehicle routes is more critical than increasing capacity. Future improvements should focus on refining routing strategies rather than allocating additional capacity beyond what is necessary to meet demand.